In [1]:
#Импортируйте библиотеку pandas как pd. Прочитайте датасет bookings.csv , находящийся по ссылке 
import pandas as pd
bookings = pd.read_csv('https://stepik.org/media/attachments/lesson/360344/bookings.csv', sep = ';')

In [2]:
bookings.shape

(119390, 21)

In [3]:
#Затем запишите первые 7 строк в переменную bookings_head
bookings_head = bookings.head(7)
bookings_head

,Hotel,Is Canceled,Lead Time,arrival full date,Arrival Date Year,Arrival Date Month,Arrival Date Week Number,Arrival Date Day of Month,Stays in Weekend nights,Stays in week nights,...,Adults,Children,Babies,Meal,Country,Reserved Room Type,Assigned room type,customer type,Reservation Status,Reservation status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
5,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03
6,Resort Hotel,0,0,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-03


In [4]:
# Приведите названия колонок к нижнему регистру и замените пробелы на знак нижнего подчеркивания.
def low_register(name):
    new_name = name.lower()
    return new_name
bookings = bookings.rename(columns = low_register)
def replace_name(name):
    new_name = name.replace(' ', '_')
    return new_name
bookings = bookings.rename(columns = replace_name)
bookings.head()
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

In [5]:
# На сколько ночей (stays_total_nights)  в среднем бронируют отели типа City Hotel? Resort Hotel? 
bookings_new = bookings.groupby('hotel', as_index = False)\
    .agg({'stays_total_nights':'mean'})\
    .round(2)
bookings_new


,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


In [6]:
# Теперь проанализируйте даты запланированного прибытия (arrival_date_year).
# На какой месяц чаще всего оформляли бронь в 2016 году? Изменился ли самый популярный месяц в 2017?
bookings.query("is_canceled == 1 and hotel == 'City Hotel'")\
    .groupby('arrival_date_year')['arrival_date_month'].value_counts()

arrival_date_year  arrival_date_month
2015               September             1543
                   October               1321
                   August                1232
                   July                   939
                   December               668
                   November               301
2016               October               1947
                   June                  1720
                   September             1567
                   April                 1539
                   May                   1436
                   November              1360
                   August                1247
                   March                 1108
                   December              1072
                   July                  1043
                   February               930
                   January                438
2017               May                   2217
                   April                 1926
                   June                  1

In [7]:
#Посмотрите на числовые характеристики трёх колонок: adults, children и babies. Какая из них имеет наибольшее среднее значение?
bookings[['adults', 'children', 'babies']].mean().round(2)

adults      1.86
children    0.10
babies      0.01
dtype: float64

In [8]:
#Создайте колонку total_kids, объединив столбцы children и babies. 
#Для отелей какого типа среднее значение переменной оказалось наибольшим?
bookings['total_kids'] = bookings.children + bookings.babies
bookings.groupby('hotel')\
    .agg({'total_kids':'mean'})\
    .round(2)

,total_kids
hotel,
City Hotel,0.10
Resort Hotel,0.14


    Не все бронирования завершились успешно (is_canceled), поэтому можно посчитать, сколько клиентов было потеряно в процессе. Churn rate (отток, коэффициент оттока) – это процент подписчиков (например, на push-уведомления от сайта), которые отписались от канала коммуникации, отказались от услуг сервиса в течение определенного периода времени. Иными словами, представляет собой отношение количества ушедших пользователей к общему количеству пользователей, выраженное в процентах.

    Создайте переменную has_kids, которая принимает значение True, если клиент при бронировании указал хотя бы одного ребенка (total_kids), в противном случае – False. Далее проверьте, среди какой группы пользователей показатель оттока выше.

In [9]:
bookings.query('is_canceled == 1').is_canceled.value_counts()
bookings.is_canceled.value_counts()
churn_rate = 44224 / (75166 + 44224)
churn_rate

0.37041628277075134

In [10]:
bookings['has_kids'] = bookings['total_kids'] >= 1

In [11]:
bookings.query('has_kids == True').is_canceled.value_counts()
churn_rate = 3259 / (3259 + 6073) * 100
round(churn_rate, 2)

34.92

In [12]:
bookings.query('has_kids == False').is_canceled.value_counts()
churn_rate = 40965 / (40965 + 69093) * 100
round(churn_rate, 2)

37.22